# Проверка серверного API

In [1]:
import dill
import urllib.request
import json
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

## Исходные данные для тестирования

In [2]:
X_test = pd.read_pickle('data/X_test.pkl.zip')
y_test = pd.read_pickle('data/y_test.pkl.zip')

## Проверка работоспособности

In [3]:
# Запрос на базе данных первых двух клиентов тестового датасета
first2 = X_test.index.unique()[:2]
request_json = X_test.loc[first2].reset_index().to_dict('list')

In [4]:
def get_response(request_json):    
    myurl = "http://127.0.0.1:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(request_json)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())

In [5]:
get_response(request_json)

{'predictions': [1, 1], 'success': True}

## Prediction

In [6]:
# Запрос на базе данных части клиентов тестового датасета
clients = X_test.index.unique()[:200]
request_json = X_test.loc[clients].reset_index().to_dict('list')

In [7]:
%%time
request_json = X_test.loc[clients].reset_index().to_dict('list')
y_pred = get_response(request_json)['predictions']
print(classification_report(y_test.loc[clients], y_pred))

              precision    recall  f1-score   support

           0       0.54      0.49      0.51        41
           1       0.71      0.77      0.74        70
           2       0.64      0.89      0.74        18
           3       0.58      0.51      0.54        71

    accuracy                           0.63       200
   macro avg       0.62      0.66      0.63       200
weighted avg       0.62      0.63      0.62       200

CPU times: user 625 ms, sys: 12.8 ms, total: 638 ms
Wall time: 1.09 s
